In [23]:
import pandas as pd
import json
import re
import csv
import time
from tqdm import tqdm
from ollama import chat


input_file = '/home/ali/Review_Quality_Benchmark/data/processed/openreview_iclr2024_v8.json'
# Load data
df = pd.read_json(input_file)

llm_fields = [
    "llm_length_effort", "llm_lexical_diversity", "llm_questions_raised",
    "llm_citation_usage", "llm_sentiment_polarity", "llm_politeness", "llm_hedging",
    "llm_specificity", "llm_domain_terms", "llm_relevance_alignment",
    "llm_readability", "llm_overall_quality", "llm_overall_score_100"
]

# Check for missing fields and add them if not present
for field in llm_fields:
    if field not in df.columns:
        df[field] = pd.NA

# Pattern to extract JSON block
pattern = re.compile(r"<review_assessment>\s*(\{.*?\})\s*</review_assessment>", re.DOTALL)

# Define prompt template
template = """# REVIEW-QUALITY JUDGE

## 0 — ROLE  
You are **ReviewInspector-LLM**, a rigorous, impartial meta-reviewer.  
Your goal is to grade the quality of a single peer-review against a rich set of predefined criteria and to provide concise, actionable feedback.

## 1 — INPUTS  
Title : {title}  
Abstract : {abstract}  
Review: {review_text}

## 2 — EVALUATION CRITERIA  
Return **only** the scale value or label at right (no rationale text).

| # | Criterion | Allowed scale / label |
|---|-----------|-----------------------|
| 1 | **Length & Effort**             | integer **0-5** |
| 2 | **Lexical Diversity (TTR)**     | integer **0-5** |
| 3 | **Number of Questions Raised**  | non-negative **integer** |
| 4 | **Citation Usage**              | **yes / no** |
| 5 | **Sentiment Polarity**          | **negative / neutral / positive** |
| 6 | **Politeness**                  | **polite / neutral / impolite** |
| 7 | **Hedging / Uncertainty**       | **No Hedging / Minimal / Moderate / Heavy / Extreme** |
| 8 | **Specificity**                 | **very specific / somewhat specific / neutral / broad / very broad** |
| 9 | **Use of Domain-Specific Terms**| integer **0-5** |
|10 | **Relevance Alignment**         | integer **0-5** |
|11 | **Readability**                 | integer **0-5** |
|12 | **Overall Quality**             | integer **0-100** |

## 3 — SCORING GUIDELINES  
For 0-5 scales: 5 = Outstanding, 4 = Strong, 3 = Adequate, 2 = Weak, 1 = Very weak, 0 = Absent/irrelevant.

## 4 — ANALYSIS & COMPUTATION (silent)  
1. Read and comprehend the review text.  
2. Compute raw metrics (word count, TTR, sentiment, FK grade, counts, etc.) or otherwise quantify qualitative aspects.  
3. Map raw metrics to the scales above.

## 5 — OUTPUT FORMAT (strict)  
Return **exactly one** JSON block wrapped in the tag below — **no comments or extra text**.

```json
<review_assessment>
{{
  "paper_title": "{title}",
  "criteria": {{
    "length_effort":       ...,
    "lexical_diversity":   ...,
    "questions_raised":    ...,
    "citation_usage":      ...,
    "sentiment_polarity":  ...,
    "politeness":          ...,
    "hedging":             ...,
    "specificity":         ...,
    "domain_terms":        ...,
    "relevance_alignment": ...,
    "readability":         ...,
    "overall_quality":     ...
  }},
  "overall_score_100": ...
}}
</review_assessment>
```
"""

In [35]:
# Process each row
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Scoring with LLM"):
    # Skip if all llm fields are already filled
    if all(pd.notna(row.get(field, pd.NA)) for field in llm_fields):
        continue
    if idx >= 800:
        break

    prompt = template.format(
        title=row['submission_title'],
        abstract=row['submission_abstract'],
        review_text=row['total_review']
    )

    for attempt in range(5):
        try:
            response = chat("llama3.2", messages=[{'role': 'user', 'content': prompt}])
            content = response['message']['content']
            match = pattern.search(content)
            if not match:
                raise ValueError("No JSON block found")

            parsed = json.loads(match.group(1))
            print(parsed["overall_score_100"])
            for key, val in parsed["criteria"].items():
                df.at[idx, f"llm_{key}"] = val
            df.at[idx, "llm_overall_score_100"] = parsed["overall_score_100"]

            # Save after every successful row
            # df.to_csv(input_file, index=False, quoting=csv.QUOTE_ALL)
            break

        except Exception as e:
            print(f"❌ Error at row {idx}, attempt {attempt + 1}: {e}")
            # time.sleep(0.5)

Scoring with LLM:   0%|          | 1/28028 [00:00<1:05:30,  7.13it/s]

❌ Error at row 24, attempt 1: No JSON block found
❌ Error at row 24, attempt 2: No JSON block found
❌ Error at row 24, attempt 3: No JSON block found
❌ Error at row 24, attempt 4: No JSON block found


Scoring with LLM:   0%|          | 25/28028 [00:06<1:55:07,  4.05it/s]

❌ Error at row 24, attempt 5: No JSON block found


Scoring with LLM:   0%|          | 52/28028 [00:07<1:01:50,  7.54it/s]

...

92
❌ Error at row 798, attempt 1: No JSON block found


Scoring with LLM:   3%|▎         | 799/28028 [23:05<29:45:39,  3.93s/it]

85
❌ Error at row 799, attempt 1: No JSON block found


Scoring with LLM:   3%|▎         | 800/28028 [23:10<13:08:40,  1.74s/it]

80


In [38]:
df

,submission_id,submission_number,submission_creation_date,submission_authors,submission_title,submission_abstract,reviewer,creation_date,last_modification_date,review_rating,...,llm_citation_usage,llm_sentiment_polarity,llm_politeness,llm_hedging,llm_specificity,llm_domain_terms,llm_relevance_alignment,llm_readability,llm_overall_quality,llm_overall_score_100
0,zzv4Bf50RW,1647,1695102158671,"[~Chunghyun_Park1, ~Seungwook_Kim2, ~Jaesik_Pa...",Learning SO(3)-Invariant Correspondence via Po...,Establishing accurate dense 3D correspondences...,Reviewer_eS3u,1698243150596,1699636093263,6,...,True,neutral,polite,Minimal,3,4,5,4,85,85
1,zzv4Bf50RW,1647,1695102158671,"[~Chunghyun_Park1, ~Seungwook_Kim2, ~Jaesik_Pa...",Learning SO(3)-Invariant Correspondence via Po...,Establishing accurate dense 3D correspondences...,Reviewer_jP4i,1698652503617,1699636093190,5,...,yes,positive,polite,Minimal,4,3,5,4,85,85
2,zzv4Bf50RW,1647,1695102158671,"[~Chunghyun_Park1, ~Seungwook_Kim2, ~Jaesik_Pa...",Learning SO(3)-Invariant Correspondence via Po...,Establishing accurate dense 3D correspondences...,Reviewer_wiS9,1698706547448,1699636093122,3,...,True,neutral,polite,No Hedging,somewhat specific,3,4,4,92,92
3,zzv4Bf50RW,1647,1695102158671,"[~Chunghyun_Park1, ~Seungwook_Kim2, ~Jaesik_Pa...",Learning SO(3)-Invariant Correspondence via Po...,Establishing accurate dense 3D correspondences...,Reviewer_a6Ps,1698768293694,1699636092942,5,...,True,positive,polite,No Hedging,neutral,4,3,5,83,84
4,zzv4Bf50RW,1647,1695102158671,"[~Chunghyun_Park1, ~Seungwook_Kim2, ~Jaesik_Pa...",Learning SO(3)-Invariant Correspondence via Po...,Establishing accurate dense 3D correspondences...,Reviewer_Frem,1699350072271,1699636092872,5,...,True,neutral,neutral,Minimal,3,4,3,4,73,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28023,014CgNPAGy,2200,1695179071455,"[~Bochen_Lyu1, ~Zhanxing_Zhu1]",On the Role of Momentum in the Implicit Bias o...,Momentum is a widely adopted and crucial modif...,Reviewer_vt7i,1698673110283,1699636153803,5,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
28024,014CgNPAGy,2200,1695179071455,"[~Bochen_Lyu1, ~Zhanxing_Zhu1]",On the Role of Momentum in the Implicit Bias o...,Momentum is a widely adopted and crucial modif...,Reviewer_oaZ7,1698928691830,1699636153728,3,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
28025,0074qaufB6,5962,1695403263602,"[~Subrata_Kumar_Biswas1, ~Bashima_Islam1]",InfoNet: Missing Information Retrieval in Mult...,Faulty sensors in a multiple input stream setu...,Reviewer_fMm6,1698618130371,1699636636496,1,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
28026,0074qaufB6,5962,1695403263602,"[~Subrata_Kumar_Biswas1, ~Bashima_Islam1]",InfoNet: Missing Information Retrieval in Mult...,Faulty sensors in a multiple input stream setu...,Reviewer_tZQw,1698807944071,1699636636378,3,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [37]:
df.to_json('/home/ali/Review_Quality_Benchmark/data/processed/iclr2024_llm.json', orient='records', lines=True)
